## Aulia Nur Fitriani ##
## 21083010051 ##
## Data Mining 1 A ##

In [2]:
# Import library yang dibutuhkan
import pandas as pd
import numpy as np

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# Load dataset
data = pd.read_csv('datacategorical.csv')
data 

,id,Age,Income,Student,Credit_rating,Class : buys_computer
0,1,<=30,High,No,Fair,No
1,2,<=30,High,No,Excelent,No
2,3,31-40,High,No,Fair,Yes
3,4,>40,Medium,No,Fair,Yes
4,5,>40,Low,Yes,Fair,Yes
5,6,>40,Low,Yes,Excelent,No
6,7,31-40,Low,Yes,Excelent,Yes
7,8,<=30,Medium,No,Fair,No
8,9,<=30,Low,Yes,Fair,Yes
9,10,>40,Medium,Yes,Fair,Yes


In [4]:
data = data.drop(columns = ['id'])
data

,Age,Income,Student,Credit_rating,Class : buys_computer
0,<=30,High,No,Fair,No
1,<=30,High,No,Excelent,No
2,31-40,High,No,Fair,Yes
3,>40,Medium,No,Fair,Yes
4,>40,Low,Yes,Fair,Yes
5,>40,Low,Yes,Excelent,No
6,31-40,Low,Yes,Excelent,Yes
7,<=30,Medium,No,Fair,No
8,<=30,Low,Yes,Fair,Yes
9,>40,Medium,Yes,Fair,Yes


In [5]:
# Preprocess data
def preprocess_dataframe(data):
    # Convert categorical to numerical data
    # Age
    data['Age'] = data['Age'].apply(lambda x: 0 if x=='<=30' else 1 if x=='31..40' else 2)

    # Income
    data['Income'] = data['Income'].apply(lambda x: 0 if x=='High' else 1 if x=='Medium' else 2)

    # Student
    data['Student'] = data['Student'].apply(lambda x: 0 if x=='No' else 1)

    # Credit_rating
    data['Credit_rating'] = data['Credit_rating'].apply(lambda x: 0 if x=='Fair' else 1)

    # Class : buys_computer
    if 'Class : buys_computer' in data.columns:
        data['Class : buys_computer'] = data['Class : buys_computer'].apply(lambda x: 0 if x=='No' else 1)

        # Set X & y
        y = data['Class : buys_computer'].to_numpy()
        X = data.drop(['Class : buys_computer'], axis=1).to_numpy()
    
        return X, y
    else:
        X = data.to_numpy()
        
        return X, None

# Execute preprocessing data
X, y = preprocess_dataframe(data)
print(X)
print(y)

[[0 0 0 0]
 [0 0 0 1]
 [2 0 0 0]
 [2 1 0 0]
 [2 2 1 0]
 [2 2 1 1]
 [2 2 1 1]
 [0 1 0 0]
 [0 2 1 0]
 [2 1 1 0]
 [0 1 1 1]
 [2 1 0 1]
 [2 0 1 0]
 [2 1 0 1]]
[0 0 1 1 1 0 1 0 1 1 1 1 1 0]


**1. Perhitungan probabilitas tiap nilai nominal pada label kelas**

In [6]:
# Membuat fungsi untuk menghitung probabilitas tiap nilai nominal pada label kelas
def calculate_class_probabilities(data):
    # Menghitung jumlah baris dalam dataset
    num_rows = len(data)
    
    # Menghitung jumlah setiap nilai pada label kelas
    class_counts = data['Class : buys_computer'].value_counts()
    
    # Membuat dictionary untuk menyimpan probabilitas tiap nilai pada label kelas
    class_probabilities = {}
    
    # Menghitung probabilitas tiap nilai pada label kelas
    for class_value, count in class_counts.items():
        class_probabilities[class_value] = count / float(num_rows)
    
    return class_probabilities

# Menghitung probabilitas tiap nilai pada label kelas untuk dataset yang telah dimuat
class_probabilities = calculate_class_probabilities(data)
print(class_probabilities)

{1: 0.6428571428571429, 0: 0.35714285714285715}


**2. Perhitungan probabilitas tiap nilai fitur/variabel prediktor (case) terhadap label kelas**

In [10]:
def calculate_probabilities(data, predictor):
    probabilities = {}
    
    # Get unique values of predictor variable
    predictor_values = data[predictor].unique()
    
    # Calculate the probabilities for each predictor value
    for predictor_value in predictor_values:
        predictor_value_count = len(data[data[predictor] == predictor_value])
        probabilities[predictor_value] = predictor_value_count / len(data)
    
    return probabilities

# Calculate conditional probabilities for each predictor variable
predictors = ['Age', 'Income', 'Student', 'Credit_rating']
class_labels = ['1', '0'] # Specify the class labels for which probabilities are calculated

probabilities = {}
for predictor in predictors:
    predictor_probabilities = {}
    for class_label in class_labels:
        class_label_data = data[data['Class : buys_computer'] == class_label]
        conditional_probabilities = calculate_probabilities(class_label_data, predictor)
        predictor_probabilities[class_label] = conditional_probabilities

    probabilities[predictor] = predictor_probabilities

# Print the probabilities for each predictor variable
for predictor, predictor_probabilities in probabilities.items():
    print("Conditional probabilities for", predictor)
    for value, class_label_probabilities in predictor_probabilities.items():
        print(value, ": ", class_label_probabilities)
    print()

Conditional probabilities for Age
1 :  {}
0 :  {}

Conditional probabilities for Income
1 :  {}
0 :  {}

Conditional probabilities for Student
1 :  {}
0 :  {}

Conditional probabilities for Credit_rating
1 :  {}
0 :  {}



**3. Perkalian tiap probabilitas nilai fitur/variabel prediktor terhadap label kelas dengan probabilitas tiap nilai nominal pada label kelas**

In [9]:
# Get the unique class labels
class_labels = data['Class : buys_computer'].unique()

# Calculate the probabilities for each predictor value for each class label
probabilities = {}
for predictor in data.columns[:-1]:
    predictor_probabilities = {}
    for class_label in class_labels:
        class_label_data = data[data['Class : buys_computer'] == class_label]
        predictor_values = data[predictor].unique()
        for predictor_value in predictor_values:
            predictor_value_count = len(class_label_data[class_label_data[predictor] == predictor_value])
            total_class_label_count = len(class_label_data)
            predictor_probabilities[(predictor_value, class_label)] = predictor_value_count / total_class_label_count
    probabilities[predictor] = predictor_probabilities

# Calculate the total probabilities for each class label
class_label_probabilities = {}
for class_label in class_labels:
    class_label_data = data[data['Class : buys_computer'] == class_label]
    class_label_count = len(class_label_data)
    total_count = len(data)
    class_label_probabilities[class_label] = class_label_count / total_count

# Calculate the final probabilities for each predictor value for each class label
final_probabilities = {}
for predictor in probabilities:
    predictor_probabilities = {}
    for class_label in class_labels:
        class_label_probability = class_label_probabilities[class_label]
        for predictor_value in data[predictor].unique():
            predictor_value_probability = probabilities[predictor][(predictor_value, class_label)]
            final_probability = class_label_probability * predictor_value_probability
            predictor_probabilities[(predictor_value, class_label)] = final_probability
    final_probabilities[predictor] = predictor_probabilities

# Print the final probabilities for each predictor value for each class label
for predictor in final_probabilities:
    print("probabilitas untuk:", predictor)
    for class_label in class_labels:
        print("Class label:", class_label)
        for predictor_value in data[predictor].unique():
            probability = final_probabilities[predictor][(predictor_value, class_label)]
            print(predictor_value, ": ", probability)
        print()
    print()

probabilitas untuk: Age
Class label: 0
0 :  0.21428571428571427
2 :  0.14285714285714288

Class label: 1
0 :  0.14285714285714285
2 :  0.5


probabilitas untuk: Income
Class label: 0
0 :  0.14285714285714288
1 :  0.14285714285714288
2 :  0.07142857142857144

Class label: 1
0 :  0.14285714285714285
1 :  0.2857142857142857
2 :  0.2142857142857143


probabilitas untuk: Student
Class label: 0
0 :  0.28571428571428575
1 :  0.07142857142857144

Class label: 1
0 :  0.2142857142857143
1 :  0.4285714285714286


probabilitas untuk: Credit_rating
Class label: 0
0 :  0.14285714285714288
1 :  0.21428571428571427

Class label: 1
0 :  0.4285714285714286
1 :  0.2142857142857143




**4. Membandingkan nilai probabilitas yang besar sebagai hasil prediksi label kelas dari record/instance**

In [41]:
def preprocess_dataframe_invert(data):
    # Convert categorical to numerical data
    # Age
    data['Age'] = data['Age'].apply(lambda x: 0 if x=='<=30' else 1 if x=='31-40' else 2)

    # Income
    data['Income'] = data['Income'].apply(lambda x: 0 if x=='High' else 1 if x=='Medium' else 2)

    # Student
    data['Student'] = data['Student'].apply(lambda x: 0 if x=='No' else 1)

    # Credit_rating
    data['Credit_rating'] = data['Credit_rating'].apply(lambda x: 0 if x=='Fair' else 1)
    
    # Target
    data['Class : buys_computer'] = data['Class : buys_computer'].apply(lambda x: 'N' if x==0 else 'Y')
    
    return data

# Define the target variable values
choosen_class = pd.Series([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1])

# Execute preprocessing data
df_test['Class : buys_computer'] = choosen_class

df_test = preprocess_dataframe_invert(df_test)
df_test

,Age,Income,Student,Credit_rating,Class : buys_computer
0,2,2,1,1,N
1,2,2,1,1,Y
2,2,2,1,1,N
3,2,2,1,1,N
4,2,2,1,1,N
5,2,2,1,1,Y
6,2,2,1,1,Y
7,2,2,1,1,N
8,2,2,1,1,N
9,2,2,1,1,N


**5. Pembagian data training dan data testing menggunakan percentage split (misal: 70:30, 80:20, dan 90:10)**

In [37]:
import pandas as pd
import numpy as np
import math

def train_test_split(data, test_size=0.2, random_state=None):
    # Membuat array indeks acak untuk data
    np.random.seed(random_state)
    shuffled_indices = np.random.permutation(len(data))
    # Menghitung jumlah data testing berdasarkan test_size
    test_size = math.ceil(len(data) * test_size)
    
    # Memilih indeks data training dan data testing
    test_indices = shuffled_indices[:test_size]
    train_indices = shuffled_indices[test_size:]
    
    # Membagi data menjadi data training dan data testing
    train_data = data.iloc[train_indices]
    test_data = data.iloc[test_indices]
    
    return train_data, test_data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=1)
X_test = test_data.iloc[:,:-1].values
Y_test = test_data.iloc[:,-1].values
print("Training data :", train_data)
print("\n", "Testing data :", test_data)

Training data :     Age  Income  Student  Credit_rating  Class : buys_computer
2     2       0        0              0                      1
10    0       1        1              1                      1
4     2       2        1              0                      1
1     0       0        0              1                      0
12    2       0        1              0                      1
0     0       0        0              0                      0
13    2       1        0              1                      0
9     2       1        1              0                      1
8     0       2        1              0                      1
11    2       1        0              1                      1
5     2       2        1              1                      0

 Testing data :    Age  Income  Student  Credit_rating  Class : buys_computer
3    2       1        0              0                      1
7    0       1        0              0                      0
6    2       2        1  

**6. Perhitungan performa klasifikasi menggunakan confusion matrix (Accuracy, Recall, Precision, F-measure)**

In [30]:
def prob_prior(data, Y):
    label_kelas = sorted(list(data[Y].unique()))
    prior = []
    for i in label_kelas:
        prior.append(len(data[data[Y]==i])/len(data))
    return prior

def likelihood_categorical(data, nama_var, nilai_var, Y, label):
    feat = list(data.columns)
    data = data[data[Y]==label]
    p_x_dgsyrt_y = len(data[data[nama_var]==nilai_var]) / len(data)
    return p_x_dgsyrt_y

def naive_bayes_categorical(data, X, Y):
    # mengambil nama kolom
    nama_kolom = list(data.columns)[:-1]
    # menghitung prob prior
    prior = prob_prior(data, Y)
    Y_pred = []
    # lopping pada setiap baris data
    for x in X:
        # menghitung likelihood
        labels = sorted(list(data[Y].unique()))
        likelihood = [1]*len(labels)
        for j in range(len(labels)):
            for i in range(len(nama_kolom)):
                likelihood[j] *= likelihood_categorical(data, nama_kolom[i], x[i], Y, labels[j])
                                                                                           
        # menghitung posterior probability
        post_prob = [1]*len(labels)
        for j in range(len(labels)):
            post_prob[j] = likelihood[j] * prior[j]

        Y_pred.append(np.argmax(post_prob))
    return np.array(Y_pred)

In [32]:
Y_pred = naive_bayes_categorical(train_data, X=X_test, Y="Class : buys_computer")

In [34]:
# Menghitung true positives, false positives, true negatives, false negatives
tp = fp = tn = fn = 0
for i in range(len(Y_test)):
    if Y_test[i] == 1 and Y_pred[i] == 1:
        tp += 1
    elif Y_test[i] == 1 and Y_pred[i] == 0:
        fn += 1
    elif Y_test[i] == 0 and Y_pred[i] == 1:
        fp += 1
    elif Y_test[i] == 0 and Y_pred[i] == 0:
        tn += 1

# Menghitung accuracy, recall, precision, dan f1-score
accuracy = (tp + tn) / (tp + fp + tn + fn)
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = 2 * (precision * recall) / (precision + recall)

# Menampilkan confusion matrix dan performa klasifikasi
print(f"Confusion Matrix:\n{tn}\t{fp}\n{fn}\t{tp}\n")
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Confusion Matrix:
0	1
0	2

Accuracy: 0.67
Recall: 1.00
Precision: 0.67
F1 Score: 0.80


In [45]:
# split dataset menjadi x dan y label
x = data.drop('Class: buys_computer', axis=1)
y = data['Class: buys_computer']

# mendapatkan nilai unik
class_values = y.unique()

# hitung probabilitas bersyarat untuk setiap kelas
conditional_probabilities = {}
for feature in x.columns:
    feature_values = x[feature].unique()
    for feature_value in feature_values:
        for class_value in class_values:
            numerator = len(x[(x[feature] == feature_value) & (y == class_value)])
            denominator = len(x[y == class_value])
            conditional_probability = numerator / denominator
            if feature not in conditional_probabilities:
                conditional_probabilities[feature] = {}
            if feature_value not in conditional_probabilities[feature]:
                conditional_probabilities[feature][feature_value] = {}
            conditional_probabilities[feature][feature_value][class_value] = conditional_probability

# Print outputnya
for feature, feature_values in conditional_probabilities.items():
    print("Feature:", feature)
    for feature_value, class_probabilities in feature_values.items():
        print("  -", feature_value)
        for class_value, probability in class_probabilities.items():
            print("    ", class_value, ":", probability)

KeyError: "['Class: buys_computer'] not found in axis"